In [ ]:
%%javascript
$('#appmode-leave').hide();
$('#copy-binder-link').hide();
$('#visit-repo-link').hide();

In [ ]:
import ipywidgets as ipw
import json
import os
import webbrowser
import math

import glob as glob
import nbformat as nbf

from IPython import get_ipython
from IPython.display import display, Markdown, FileLink, clear_output, Javascript

label_layout = ipw.Layout(width='300px')

In [ ]:
import time
import random

import sys

from io import StringIO

from openmm.app import *
from openmm import *
from simtk.unit import *

import openmmtools as tools
import MDAnalysis as md
import nglview as ng

# Molecular Dynamics parameters

In [ ]:
out_P = ipw.Output()

params = {}
params["h2o"] = "None"
params["T"] = 300
params["P"] = 1
params["Time"] = 1
params["ts"] = 0.002

# output filename
params["thermoFile"] = "thermo.csv"
params["coordFile"] = "water.pdb"
params["trajFile"] = "traj.dcd"

def reset(btn):
    with out_P:
        out_P.clear_output()

In [ ]:
import matplotlib.pyplot as plt

t = int( time.time() * 1000.0 )
random.seed( ((t & 0xff000000) >> 24) +
             ((t & 0x00ff0000) >>  8) +
             ((t & 0x0000ff00) <<  8) +
             ((t & 0x000000ff) << 24)   )

def runMD(btn): 
    
    if params["h2o"] == "None":
        with out_P:
            print("Select a water model before running the MD")
        return
    
    ### 1. Choose a water model
    params["T"] = float(temperature.value)
    params["P"] = float(pressure.value)
    params["Time"] = float(simTime.value)
    params["ts"] = float(timestep.value)
    
    params["thermoFile"] = thermoFile.value
    params["coordFile"] = coordFile.value
    params["trajFile"] = trajFile.value
    
    ### 2. Create initial coordinates and system
    water = tools.testsystems.WaterBox(model=params["h2o"],
                                       box_edge=2.1*nanometers,
                                       ewaldErrorTolerance=1e-04)

    ### 2a. Save initial coordinates to file
    output = open(params["coordFile"], 'w')
    PDBFile.writeFile(water.topology, water.positions, output)
    output.flush()
    
    ### 3 Simulation timestep, total number of steps and thermostat/barostat parameters
    timeStep = params["ts"] * picoseconds
    totalSteps = params["Time"] * picosecond / timeStep
    frictionCoeff = 0.1/picosecond
    nupdt = 20

    ### 4. Choose an integrator
    integrator = LangevinIntegrator( params["T"] * kelvin, frictionCoeff, timeStep )
    water.system.addForce(MonteCarloBarostat( params["P"] * atmosphere , params["T"] * kelvin , nupdt ) )

    ### 5 Create a simulation object
    simulation = Simulation(water.topology, water.system, integrator)
    simulation.context.setPositions(water.positions)
    simulation.context. setVelocitiesToTemperature(params["T"] * kelvin, random.randrange(99999))

    ### 6. Set output for trajectory and thermodynamics data
    simulation.reporters.append(DCDReporter(params["trajFile"], 10))

    ### 7. Set output for thermodynamics data
    simulation.reporters.append(StateDataReporter( params["thermoFile"]
                                                 , 10
                                                 , step            = True
                                                 , time            = True
                                                 , potentialEnergy = True
                                                 , temperature     = True
                                                 , density         = True
                                                 , volume          = True
                                                 , separator       = ","
                                                 ))

    ### 7a. Set screen output to monitor the simulation progress
    simulation.reporters.append(StateDataReporter( sys.stdout
                                                 , int(totalSteps/20)
                                                 , step            = True
                                                 , progress        = True
                                                 , remainingTime   = True
                                                 , elapsedTime     = True
                                                 , separator       = " "
                                                 , totalSteps      = totalSteps
                                                 ))

    ### 8. Run MD
    with out_P:
#         simulation.minimizeEnergy()
        simulation.step(totalSteps)


In [ ]:
# interactive buttons ---
btn_calc = ipw.Button(description="Run MD", layout=ipw.Layout(width="150px"))
btn_calc.on_click(runMD)

btn_reset = ipw.Button(description="Clear output", layout=ipw.Layout(width="150px"))
btn_reset.on_click(reset)

# --- create the boxes and sliders
rows = []

w = ipw.Dropdown(
    options=['None',
             'spce', 
             'tip3p', 
             'tip4pew', 
             'tip5p'],
    value='None',
    description='Water Model'
)

def sample_select(change):
    if change['new'] == "spce":
        params["h2o"] = "spce"
    if change['new'] == "tip3p":
        params["h2o"] = "tip3p"
    if change['new'] == "tip4pew":
        params["h2o"] = "tip4pew"
    if change['new'] == "tip5p":
        params["h2o"] = "tip5p"
    
w.observe(sample_select)

rows.append(ipw.HBox([w]))

thermoFile = ipw.Text(params["thermoFile"])
coordFile = ipw.Text(params["coordFile"])
trajFile = ipw.Text(params["trajFile"])

rows.append(ipw.HBox([ipw.Label('Thermodynamic output : ',layout=label_layout),thermoFile]))
rows.append(ipw.HBox([ipw.Label('Initial coordinates : ',layout=label_layout),coordFile]))
rows.append(ipw.HBox([ipw.Label('Simulation trajectory : ',layout=label_layout),trajFile]))

simTime = ipw.Text(str(params["Time"]))
box = ipw.Box([ipw.Label('Simulation time [ps]  :  ',layout=label_layout),simTime])
rows.append(ipw.HBox([box]))

timestep = ipw.Text(str(params["ts"]))
box = ipw.Box([ipw.Label('Simulation timestep [ps]  :  ',layout=label_layout),timestep])
rows.append(ipw.HBox([box]))

temperature = ipw.Text(str(params["T"]))
box = ipw.Box([ipw.Label('Temperature [K]  :  ',layout=label_layout),temperature])
rows.append(ipw.HBox([box]))

pressure = ipw.Text(str(params["P"]))
box = ipw.Box([ipw.Label('Simulation pressure [atm]  :  ',layout=label_layout),pressure])
rows.append(ipw.HBox([box]))

# ---
reset(btn_reset)

rows.append(ipw.HBox([btn_calc,btn_reset]))
rows.append(ipw.HBox([out_P]))

ipw.VBox(rows)

# Working Notebooks

In [ ]:
##########
pfiles = ['.protectedFiles.txt' , '../.protectedFiles.txt']
for fff in pfiles:
    if os.path.isfile(fff):
        with open(fff) as f:
            protectedFiles = f.read().splitlines()
##########
def launchNotebook(filename):
    text = "   var name_of_the_notebook = '" + filename + "'"
    vv="""
    var url = window.location.href.split('/')
    var newurl = url[0] + '//'
    for (var i = 1; i < url.length - 1; i++) {
        console.log(url[i], newurl)
        newurl += url[i] + '/'
    }
    newurl += name_of_the_notebook
    window.open(newurl)
    """
    text = text + vv
    display(Javascript(text))

def openNewNotebook(btn):
    if os.path.exists(notebookeName.value):
        print("Filename exists - Please select a different name")
        return

    nb = nbf.v4.new_notebook()
    text0 = """# Click 'Edit App' to start coding"""
    text1 = """## Thermodynamic data at the beginning of the run"""
    text2 = """## Thermodynamic data at the end of the run"""
    text3 = """## Plot of the cell volume"""
    text4 = """## Animation of the trajectory"""
    
    code0 = """\
# python packages
import pandas as pd # Dataframes and reading CSV files
import numpy as np # Numerical libraries
import matplotlib.pyplot as plt # Plotting library
from lmfit import Model # Least squares fitting library
from scipy.optimize import curve_fit # Alternative curve fittting library
import MDAnalysis as md
import nglview as ng
data = pd.read_csv("thermo.csv")
"""

    code1 = """
print(data.head())
    """

    code2 = """
print(data.tail())
    """

    vizCell = """\
sys = md.Universe("water.pdb", 'traj.dcd')
view = ng.show_mdanalysis(sys, gui=True)
view.center()
view.representations = [
    {"type": "ball+stick", "params": {"sele": "all"}}
]
view
"""
    
    dataCell = """\
fig = plt.figure()
ax = fig.gca()
x = data["Time (ps)"]
y = data["Box Volume (nm^3)"]
ax.plot(x,y)
ax.set(xlabel="Time (ps)")
ax.set(ylabel="Box Volume (nm^3)")
plt.show()
"""
    
    nb['cells'] = [nbf.v4.new_markdown_cell(text0),
                   nbf.v4.new_code_cell(code0),
                   nbf.v4.new_markdown_cell(text1),
                   nbf.v4.new_code_cell(code1),
                   nbf.v4.new_markdown_cell(text2),
                   nbf.v4.new_code_cell(code2),
                   nbf.v4.new_markdown_cell(text3),
                   nbf.v4.new_code_cell(dataCell),
                   nbf.v4.new_markdown_cell(text4),
                   nbf.v4.new_code_cell(vizCell)]

    if notebookeName.value in protectedFiles or notebookeName.value in listOfFiles:
        print("File already exists, select a different filename")
    else:
        with open(notebookeName.value, 'w') as f:
            nbf.write(nb, f)
        launchNotebook(notebookeName.value)

##########
listOfFiles = []
files = glob.glob1("./","*.ipynb")
for f in files:
    if f in protectedFiles:
        continue
    listOfFiles.append(f)

def dropdown_filesHandler(change):
    for i in range(0,len(listOfFiles)):
        if listOfFiles[i] == change.new:
            oldNotebookeName[0] = listOfFiles[i]

def createMenuFiles(data):
    option_list = ["Choose one"]
    option_list.extend(data)

    dropdown = ipw.Dropdown(description="", options=option_list, layout=ipw.Layout(width="300px"))
    dropdown.observe(dropdown_filesHandler, names='value')

    return dropdown

##########
oldNotebookeName = ["None"]
def openOldNotebook(btn):
    if oldNotebookeName[0] == "None":
        print("Please select a filename")
    elif oldNotebookeName[0] in protectedFiles:
        print("Please select a different filename")
    else:
        launchNotebook(oldNotebookeName[0])

##########
actions0 = []

notebookeName = ipw.Text("Empty.ipynb")

btn_new = ipw.Button(description="Create a new notebook", layout=label_layout)
btn_new.on_click(openNewNotebook)

btn_old = ipw.Button(description="Open an old notebook", layout=label_layout)
btn_old.on_click(openOldNotebook)

actions0.append(ipw.HBox([btn_new,notebookeName]))
actions0.append(ipw.HBox([btn_old,createMenuFiles(listOfFiles)]))
ipw.VBox(actions0)